In [63]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm

In [99]:
#Inception V3
import os

from tensorflow.keras import layers
from tensorflow.keras import Model

from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = None)

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
    layer.trainable = False
# pre_trained_model.summary()

last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 7, 7, 768)


In [107]:
from tensorflow.keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification  
x = layers.Dense(81, activation='softmax')(x)

model = Model( pre_trained_model.input, x) 

model.compile(loss='categorical_crossentropy',optimizer = RMSprop(lr=0.0001),metrics=['accuracy'])

In [55]:
# read csv with the predictions
train = pd.read_csv('train_labels.csv')
# look at the data format
print(train.head())

      img_name  label
0  train_1.jpg     21
1  train_2.jpg     29
2  train_3.jpg     17
3  train_4.jpg     21
4  train_5.jpg     50


In [56]:
# perform one hot encoding
y=train['label'].values
y = to_categorical(y)
print(y)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [57]:
# get validation set
X_train, X_test, y_train, y_test = train_test_split(train["img_name"], y, random_state=42, test_size=0.2)

In [75]:
INPUT_SHAPE = (150,150,3)

In [77]:
# read the images and get them in an array
def gen_train():
    while True:
        for x, y in zip(X_train, y_train):
            img = image.load_img("train_set/train_set/"+x, target_size=INPUT_SHAPE, grayscale=False)
            img = image.img_to_array(img)
            img = img/255
            yield (np.array([img]),np.array([y]))

In [78]:
# read the images and get them in an array
def gen_validation():
    while True:
        for x,y in zip(X_test, y_test):
            img = image.load_img("train_set/train_set/"+x, target_size=INPUT_SHAPE, grayscale=False)
            img = image.img_to_array(img)
            img = img/255
            yield (np.array([img]),np.array([y]))

In [103]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d_282 (Conv2D)             (None, 74, 74, 32)   864         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_282 (BatchN (None, 74, 74, 32)   96          conv2d_282[0][0]                 
__________________________________________________________________________________________________
activation_282 (Activation)     (None, 74, 74, 32)   0           batch_normalization_282[0][0]    
______________________________________________________________________________________________

__________________________________________________________________________________________________
mixed0 (Concatenate)            (None, 16, 16, 256)  0           activation_287[0][0]             
                                                                 activation_289[0][0]             
                                                                 activation_292[0][0]             
                                                                 activation_293[0][0]             
__________________________________________________________________________________________________
conv2d_297 (Conv2D)             (None, 16, 16, 64)   16384       mixed0[0][0]                     
__________________________________________________________________________________________________
batch_normalization_297 (BatchN (None, 16, 16, 64)   192         conv2d_297[0][0]                 
__________________________________________________________________________________________________
activation

__________________________________________________________________________________________________
batch_normalization_301 (BatchN (None, 16, 16, 64)   192         conv2d_301[0][0]                 
__________________________________________________________________________________________________
batch_normalization_303 (BatchN (None, 16, 16, 64)   192         conv2d_303[0][0]                 
__________________________________________________________________________________________________
batch_normalization_306 (BatchN (None, 16, 16, 96)   288         conv2d_306[0][0]                 
__________________________________________________________________________________________________
batch_normalization_307 (BatchN (None, 16, 16, 64)   192         conv2d_307[0][0]                 
__________________________________________________________________________________________________
activation_301 (Activation)     (None, 16, 16, 64)   0           batch_normalization_301[0][0]    
__________

batch_normalization_319 (BatchN (None, 7, 7, 128)    384         conv2d_319[0][0]                 
__________________________________________________________________________________________________
activation_314 (Activation)     (None, 7, 7, 128)    0           batch_normalization_314[0][0]    
__________________________________________________________________________________________________
activation_319 (Activation)     (None, 7, 7, 128)    0           batch_normalization_319[0][0]    
__________________________________________________________________________________________________
average_pooling2d_30 (AveragePo (None, 7, 7, 768)    0           mixed3[0][0]                     
__________________________________________________________________________________________________
conv2d_312 (Conv2D)             (None, 7, 7, 192)    147456      mixed3[0][0]                     
__________________________________________________________________________________________________
conv2d_315

__________________________________________________________________________________________________
batch_normalization_322 (BatchN (None, 7, 7, 192)    576         conv2d_322[0][0]                 
__________________________________________________________________________________________________
batch_normalization_325 (BatchN (None, 7, 7, 192)    576         conv2d_325[0][0]                 
__________________________________________________________________________________________________
batch_normalization_330 (BatchN (None, 7, 7, 192)    576         conv2d_330[0][0]                 
__________________________________________________________________________________________________
batch_normalization_331 (BatchN (None, 7, 7, 192)    576         conv2d_331[0][0]                 
__________________________________________________________________________________________________
activation_322 (Activation)     (None, 7, 7, 192)    0           batch_normalization_322[0][0]    
__________

activation_341 (Activation)     (None, 7, 7, 192)    0           batch_normalization_341[0][0]    
__________________________________________________________________________________________________
mixed6 (Concatenate)            (None, 7, 7, 768)    0           activation_332[0][0]             
                                                                 activation_335[0][0]             
                                                                 activation_340[0][0]             
                                                                 activation_341[0][0]             
__________________________________________________________________________________________________
conv2d_346 (Conv2D)             (None, 7, 7, 192)    147456      mixed6[0][0]                     
__________________________________________________________________________________________________
batch_normalization_346 (BatchN (None, 7, 7, 192)    576         conv2d_346[0][0]                 
__________

Non-trainable params: 8,975,264
__________________________________________________________________________________________________


In [ ]:
model.fit_generator(gen_train(), steps_per_epoch=len(X_train), epochs=10, verbose=1, validation_data=gen_validation(), validation_steps=len(X_test))

# Baseline Model

In [ ]:
# baseline model
model = Sequential()
model.add(Conv2D(128, kernel_size=(3, 3),activation='relu',input_shape=INPUT_SHAPE))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5)),
model.add(Dense(512, activation='relu')),
model.add(Dense(81, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [49]:
# fit the model 
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Train on 24489 samples, validate on 6123 samples
Epoch 1/10


 5888/24489 [======>.......................] - ETA: 6:22 - loss: 4.4273 - accuracy: 0.0000e+ - ETA: 3:46 - loss: 4.4579 - accuracy: 0.0000e+ - ETA: 2:53 - loss: 4.4468 - accuracy: 0.0104   - ETA: 2:27 - loss: 4.4306 - accuracy: 0.00 - ETA: 2:10 - loss: 4.4281 - accuracy: 0.00 - ETA: 2:00 - loss: 4.4272 - accuracy: 0.00 - ETA: 1:52 - loss: 4.4279 - accuracy: 0.00 - ETA: 1:46 - loss: 4.4247 - accuracy: 0.00 - ETA: 1:42 - loss: 4.4242 - accuracy: 0.00 - ETA: 1:38 - loss: 4.4209 - accuracy: 0.00 - ETA: 1:36 - loss: 4.4171 - accuracy: 0.00 - ETA: 1:34 - loss: 4.4155 - accuracy: 0.00 - ETA: 1:32 - loss: 4.4127 - accuracy: 0.00 - ETA: 1:30 - loss: 4.4092 - accuracy: 0.00 - ETA: 1:28 - loss: 4.4083 - accuracy: 0.01 - ETA: 1:27 - loss: 4.4073 - accuracy: 0.00 - ETA: 1:26 - loss: 4.4076 - accuracy: 0.00 - ETA: 1:25 - loss: 4.4077 - accuracy: 0.00 - ETA: 1:24 - loss: 4.4067 - accuracy: 0.00 - ETA: 1:23 - loss: 4.4058 - accuracy: 0.01 - ETA: 1:23 - loss: 4.4038 - accuracy: 0.01 - ETA: 1:23 - loss:

23840/24489 [============================>.] - ETA: 19s - loss: 4.3251 - accuracy: 0.024 - ETA: 19s - loss: 4.3250 - accuracy: 0.024 - ETA: 19s - loss: 4.3250 - accuracy: 0.024 - ETA: 19s - loss: 4.3247 - accuracy: 0.024 - ETA: 19s - loss: 4.3245 - accuracy: 0.024 - ETA: 19s - loss: 4.3241 - accuracy: 0.024 - ETA: 19s - loss: 4.3238 - accuracy: 0.024 - ETA: 18s - loss: 4.3233 - accuracy: 0.024 - ETA: 18s - loss: 4.3233 - accuracy: 0.024 - ETA: 18s - loss: 4.3230 - accuracy: 0.024 - ETA: 18s - loss: 4.3227 - accuracy: 0.024 - ETA: 18s - loss: 4.3225 - accuracy: 0.024 - ETA: 18s - loss: 4.3221 - accuracy: 0.024 - ETA: 18s - loss: 4.3222 - accuracy: 0.024 - ETA: 18s - loss: 4.3218 - accuracy: 0.024 - ETA: 18s - loss: 4.3216 - accuracy: 0.024 - ETA: 18s - loss: 4.3215 - accuracy: 0.024 - ETA: 18s - loss: 4.3215 - accuracy: 0.024 - ETA: 17s - loss: 4.3212 - accuracy: 0.024 - ETA: 17s - loss: 4.3208 - accuracy: 0.024 - ETA: 17s - loss: 4.3203 - accuracy: 0.024 - ETA: 17s - loss: 4.3198 - acc

24489/24489 [==============================] - ETA: 1s - loss: 4.2877 - accuracy: 0.02 - ETA: 1s - loss: 4.2876 - accuracy: 0.02 - ETA: 1s - loss: 4.2873 - accuracy: 0.02 - ETA: 1s - loss: 4.2869 - accuracy: 0.02 - ETA: 1s - loss: 4.2862 - accuracy: 0.02 - ETA: 1s - loss: 4.2861 - accuracy: 0.02 - ETA: 1s - loss: 4.2860 - accuracy: 0.02 - ETA: 1s - loss: 4.2857 - accuracy: 0.02 - ETA: 1s - loss: 4.2857 - accuracy: 0.02 - ETA: 0s - loss: 4.2854 - accuracy: 0.02 - ETA: 0s - loss: 4.2857 - accuracy: 0.02 - ETA: 0s - loss: 4.2861 - accuracy: 0.02 - ETA: 0s - loss: 4.2858 - accuracy: 0.02 - ETA: 0s - loss: 4.2856 - accuracy: 0.02 - ETA: 0s - loss: 4.2854 - accuracy: 0.02 - ETA: 0s - loss: 4.2851 - accuracy: 0.02 - ETA: 0s - loss: 4.2850 - accuracy: 0.02 - ETA: 0s - loss: 4.2847 - accuracy: 0.02 - ETA: 0s - loss: 4.2845 - accuracy: 0.02 - ETA: 0s - loss: 4.2849 - accuracy: 0.02 - 76s 3ms/step - loss: 4.2849 - accuracy: 0.0295 - val_loss: 4.1334 - val_accuracy: 0.0604
Epoch 2/10


 5952/24489 [======>.......................] - ETA: 1:00 - loss: 4.0790 - accuracy: 0.03 - ETA: 1:01 - loss: 4.2352 - accuracy: 0.01 - ETA: 1:01 - loss: 4.2243 - accuracy: 0.02 - ETA: 1:01 - loss: 4.2080 - accuracy: 0.03 - ETA: 1:01 - loss: 4.1544 - accuracy: 0.05 - ETA: 1:01 - loss: 4.1359 - accuracy: 0.05 - ETA: 1:01 - loss: 4.1251 - accuracy: 0.04 - ETA: 1:01 - loss: 4.1188 - accuracy: 0.05 - ETA: 1:02 - loss: 4.1119 - accuracy: 0.05 - ETA: 1:02 - loss: 4.1261 - accuracy: 0.05 - ETA: 1:02 - loss: 4.1249 - accuracy: 0.04 - ETA: 1:02 - loss: 4.1241 - accuracy: 0.04 - ETA: 1:02 - loss: 4.1164 - accuracy: 0.04 - ETA: 1:02 - loss: 4.1207 - accuracy: 0.04 - ETA: 1:02 - loss: 4.1105 - accuracy: 0.05 - ETA: 1:02 - loss: 4.1230 - accuracy: 0.05 - ETA: 1:02 - loss: 4.1160 - accuracy: 0.05 - ETA: 1:02 - loss: 4.1117 - accuracy: 0.05 - ETA: 1:02 - loss: 4.1039 - accuracy: 0.05 - ETA: 1:02 - loss: 4.1032 - accuracy: 0.05 - ETA: 1:02 - loss: 4.0997 - accuracy: 0.05 - ETA: 1:02 - loss: 4.1006 - ac

...

23936/24489 [============================>.] - ETA: 18s - loss: 3.2634 - accuracy: 0.180 - ETA: 17s - loss: 3.2635 - accuracy: 0.180 - ETA: 17s - loss: 3.2630 - accuracy: 0.180 - ETA: 17s - loss: 3.2625 - accuracy: 0.180 - ETA: 17s - loss: 3.2627 - accuracy: 0.180 - ETA: 17s - loss: 3.2623 - accuracy: 0.180 - ETA: 17s - loss: 3.2614 - accuracy: 0.180 - ETA: 17s - loss: 3.2615 - accuracy: 0.180 - ETA: 17s - loss: 3.2614 - accuracy: 0.180 - ETA: 17s - loss: 3.2614 - accuracy: 0.180 - ETA: 17s - loss: 3.2608 - accuracy: 0.180 - ETA: 17s - loss: 3.2606 - accuracy: 0.180 - ETA: 16s - loss: 3.2604 - accuracy: 0.180 - ETA: 16s - loss: 3.2604 - accuracy: 0.180 - ETA: 16s - loss: 3.2604 - accuracy: 0.180 - ETA: 16s - loss: 3.2605 - accuracy: 0.180 - ETA: 16s - loss: 3.2607 - accuracy: 0.180 - ETA: 16s - loss: 3.2611 - accuracy: 0.180 - ETA: 16s - loss: 3.2611 - accuracy: 0.180 - ETA: 16s - loss: 3.2609 - accuracy: 0.180 - ETA: 16s - loss: 3.2607 - accuracy: 0.180 - ETA: 16s - loss: 3.2609 - acc

24489/24489 [==============================] - ETA: 1s - loss: 3.2652 - accuracy: 0.18 - ETA: 1s - loss: 3.2650 - accuracy: 0.18 - ETA: 1s - loss: 3.2650 - accuracy: 0.18 - ETA: 1s - loss: 3.2646 - accuracy: 0.18 - ETA: 1s - loss: 3.2645 - accuracy: 0.18 - ETA: 0s - loss: 3.2653 - accuracy: 0.18 - ETA: 0s - loss: 3.2652 - accuracy: 0.18 - ETA: 0s - loss: 3.2654 - accuracy: 0.18 - ETA: 0s - loss: 3.2659 - accuracy: 0.18 - ETA: 0s - loss: 3.2655 - accuracy: 0.18 - ETA: 0s - loss: 3.2652 - accuracy: 0.18 - ETA: 0s - loss: 3.2655 - accuracy: 0.18 - ETA: 0s - loss: 3.2657 - accuracy: 0.18 - ETA: 0s - loss: 3.2663 - accuracy: 0.18 - ETA: 0s - loss: 3.2663 - accuracy: 0.18 - ETA: 0s - loss: 3.2663 - accuracy: 0.18 - ETA: 0s - loss: 3.2668 - accuracy: 0.18 - 70s 3ms/step - loss: 3.2668 - accuracy: 0.1801 - val_loss: 3.5986 - val_accuracy: 0.1429
Epoch 10/10


 5952/24489 [======>.......................] - ETA: 1:01 - loss: 3.4487 - accuracy: 0.15 - ETA: 1:02 - loss: 3.3789 - accuracy: 0.14 - ETA: 1:02 - loss: 3.2298 - accuracy: 0.15 - ETA: 1:02 - loss: 3.1423 - accuracy: 0.17 - ETA: 1:02 - loss: 3.0861 - accuracy: 0.17 - ETA: 1:03 - loss: 3.1060 - accuracy: 0.18 - ETA: 1:03 - loss: 3.1238 - accuracy: 0.19 - ETA: 1:03 - loss: 3.1296 - accuracy: 0.18 - ETA: 1:03 - loss: 3.1292 - accuracy: 0.19 - ETA: 1:03 - loss: 3.1358 - accuracy: 0.19 - ETA: 1:03 - loss: 3.1337 - accuracy: 0.19 - ETA: 1:03 - loss: 3.1379 - accuracy: 0.19 - ETA: 1:03 - loss: 3.1122 - accuracy: 0.20 - ETA: 1:03 - loss: 3.1266 - accuracy: 0.19 - ETA: 1:03 - loss: 3.1483 - accuracy: 0.18 - ETA: 1:03 - loss: 3.1480 - accuracy: 0.18 - ETA: 1:03 - loss: 3.1410 - accuracy: 0.18 - ETA: 1:03 - loss: 3.1315 - accuracy: 0.19 - ETA: 1:02 - loss: 3.1372 - accuracy: 0.19 - ETA: 1:02 - loss: 3.1333 - accuracy: 0.19 - ETA: 1:02 - loss: 3.1104 - accuracy: 0.19 - ETA: 1:03 - loss: 3.1290 - ac

23936/24489 [============================>.] - ETA: 18s - loss: 3.1596 - accuracy: 0.198 - ETA: 18s - loss: 3.1598 - accuracy: 0.198 - ETA: 18s - loss: 3.1602 - accuracy: 0.198 - ETA: 18s - loss: 3.1606 - accuracy: 0.198 - ETA: 18s - loss: 3.1604 - accuracy: 0.198 - ETA: 18s - loss: 3.1604 - accuracy: 0.198 - ETA: 18s - loss: 3.1605 - accuracy: 0.198 - ETA: 17s - loss: 3.1613 - accuracy: 0.198 - ETA: 17s - loss: 3.1618 - accuracy: 0.198 - ETA: 17s - loss: 3.1616 - accuracy: 0.198 - ETA: 17s - loss: 3.1619 - accuracy: 0.198 - ETA: 17s - loss: 3.1614 - accuracy: 0.198 - ETA: 17s - loss: 3.1615 - accuracy: 0.198 - ETA: 17s - loss: 3.1620 - accuracy: 0.198 - ETA: 17s - loss: 3.1617 - accuracy: 0.198 - ETA: 17s - loss: 3.1619 - accuracy: 0.198 - ETA: 17s - loss: 3.1622 - accuracy: 0.198 - ETA: 17s - loss: 3.1625 - accuracy: 0.198 - ETA: 16s - loss: 3.1624 - accuracy: 0.198 - ETA: 16s - loss: 3.1627 - accuracy: 0.198 - ETA: 16s - loss: 3.1634 - accuracy: 0.198 - ETA: 16s - loss: 3.1637 - acc

24489/24489 [==============================] - ETA: 1s - loss: 3.1655 - accuracy: 0.20 - ETA: 1s - loss: 3.1658 - accuracy: 0.20 - ETA: 1s - loss: 3.1659 - accuracy: 0.20 - ETA: 1s - loss: 3.1659 - accuracy: 0.20 - ETA: 1s - loss: 3.1658 - accuracy: 0.20 - ETA: 1s - loss: 3.1659 - accuracy: 0.20 - ETA: 0s - loss: 3.1661 - accuracy: 0.20 - ETA: 0s - loss: 3.1658 - accuracy: 0.20 - ETA: 0s - loss: 3.1659 - accuracy: 0.20 - ETA: 0s - loss: 3.1659 - accuracy: 0.20 - ETA: 0s - loss: 3.1656 - accuracy: 0.20 - ETA: 0s - loss: 3.1661 - accuracy: 0.20 - ETA: 0s - loss: 3.1662 - accuracy: 0.20 - ETA: 0s - loss: 3.1665 - accuracy: 0.20 - ETA: 0s - loss: 3.1665 - accuracy: 0.20 - ETA: 0s - loss: 3.1669 - accuracy: 0.20 - ETA: 0s - loss: 3.1673 - accuracy: 0.20 - 71s 3ms/step - loss: 3.1672 - accuracy: 0.2013 - val_loss: 3.5709 - val_accuracy: 0.1498


In [90]:
# get the test data 
test_image = []
j = 1
for i in tqdm(range(7653)):
    img = image.load_img('test_set/test_set/test_'+str(j)+".jpg", target_size=(28,28,3), grayscale=False)
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
    j = j+1
test = np.array(test_image)

100%|██████████| 7653/7653 [00:15<00:00, 483.15it/s]


In [92]:
# make the predictions with the test set
prediction = model.predict_classes(test)

In [93]:
# put it in a csv
sample = pd.read_csv('sample.csv')
sample['label'] = prediction
sample.to_csv('sample_cnn.csv', header=True, index=False)